# KarabukWildfire2025 – Analiz Defteri

Tek defter akışı: GEE init, AOI, Sentinel‑2 ön işleme, NDVI/NBR, dNDVI/dNBR, şiddet ve folium haritalar.
Önkoşullar: `pip install -r requirements.txt` ve bir kez `earthengine authenticate`.


In [1]:
# Importlar
import os
from IPython.display import IFrame, display
import pandas as pd

from src.pipeline import run_pipeline
from src.utils import ee_init


In [2]:
# Earth Engine proje kimliği (opsiyonel) ve erken başlatma
ee_project = 'karabukwildfire2025'
print('EE_PROJECT =', ee_project or '(not set; you may be prompted to authenticate)')
try:
    ee_init(project=ee_project)
    print('Earth Engine initialized.')
except Exception as e:
    print('Earth Engine init error:', e)


EE_PROJECT = karabukwildfire2025
Earth Engine initialized.


In [3]:
# Parametreler
pre_start, pre_end = '2025-07-10', '2025-07-25'
post_start, post_end = '2025-07-26', '2025-08-10'
aoi_path = 'data/aoi.geojson'  # yoksa varsayılan Karabük bbox kullanılır
out_dir = 'results'

print('AOI exists? ', os.path.exists(aoi_path))
print('OUT dir   : ', out_dir)
pre_start, pre_end, post_start, post_end


AOI exists?  True
OUT dir   :  results


('2025-07-10', '2025-07-25', '2025-07-26', '2025-08-10')

In [4]:
# Çalıştır
outputs = run_pipeline(
    pre_start=pre_start, pre_end=pre_end,
    post_start=post_start, post_end=post_end,
    aoi_geojson=aoi_path, out_dir=out_dir,
    project=ee_project
)
outputs


{'pre_ndvi_map': 'results\\pre_NDVI.html',
 'post_ndvi_map': 'results\\post_NDVI.html',
 'pre_nbr_map': 'results\\pre_NBR.html',
 'post_nbr_map': 'results\\post_NBR.html',
 'dndvi_map': 'results\\dNDVI.html',
 'dnbr_map': 'results\\dNBR.html',
 'severity_map': 'results\\severity.html',
 'summary_csv': 'results\\summary_stats.csv'}

In [5]:
# Haritaları defter içinde görüntüle (opsiyonel)
display(IFrame(src=outputs['pre_ndvi_map'], width='100%', height=480))
display(IFrame(src=outputs['post_ndvi_map'], width='100%', height=480))
display(IFrame(src=outputs['pre_nbr_map'], width='100%', height=480))
display(IFrame(src=outputs['post_nbr_map'], width='100%', height=480))
display(IFrame(src=outputs['dnbr_map'], width='100%', height=600))
display(IFrame(src=outputs['severity_map'], width='100%', height=600))


In [6]:
# Özet istatistikler
pd.read_csv(outputs['summary_csv'])


,metric,value
0,pre_mean_NDVI,0.556089
1,post_mean_NDVI,0.498622
2,mean_dNDVI,-0.057467
3,mean_dNBR,0.060281
